# Механізми уваги та трансформери

Одним із головних недоліків рекурентних мереж є те, що всі слова в послідовності мають однаковий вплив на результат. Це призводить до субоптимальної продуктивності стандартних моделей LSTM-енкодер-декодер для задач послідовність-у-послідовність, таких як розпізнавання іменованих сутностей та машинний переклад. Насправді, окремі слова у вхідній послідовності часто мають більший вплив на вихідні результати, ніж інші.

Розглянемо модель послідовність-у-послідовність, наприклад, машинний переклад. Вона реалізується за допомогою двох рекурентних мереж, де одна мережа (**енкодер**) стискає вхідну послідовність у прихований стан, а інша, **декодер**, розгортає цей прихований стан у перекладений результат. Проблема цього підходу полягає в тому, що фінальному стану мережі важко запам'ятати початок речення, що призводить до низької якості моделі для довгих речень.

**Механізми уваги** забезпечують спосіб зважування контекстуального впливу кожного вхідного вектора на кожне передбачення виходу RNN. Це реалізується шляхом створення "коротких шляхів" між проміжними станами вхідної RNN та вихідної RNN. Таким чином, при генерації вихідного символу $y_t$, ми враховуємо всі приховані стани входу $h_i$ з різними ваговими коефіцієнтами $\alpha_{t,i}$.

![Зображення моделі енкодер/декодер з додатковим шаром уваги](../../../../../translated_images/uk/encoder-decoder-attention.7a726296894fb567.webp)
*Модель енкодер-декодер з механізмом додаткової уваги у [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), цитовано з [цього блогу](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Матриця уваги $\{\alpha_{i,j}\}$ представляє ступінь, до якого певні вхідні слова впливають на генерацію конкретного слова у вихідній послідовності. Нижче наведено приклад такої матриці:

![Зображення прикладу вирівнювання, знайденого RNNsearch-50, взято з Bahdanau - arviz.org](../../../../../translated_images/uk/bahdanau-fig3.09ba2d37f202a6af.webp)

*Зображення взято з [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Рис.3)*

Механізми уваги є основою багатьох сучасних або близьких до сучасних досягнень у галузі обробки природної мови. Однак додавання уваги значно збільшує кількість параметрів моделі, що призводить до проблем масштабування з RNN. Основним обмеженням масштабування RNN є те, що рекурентна природа моделей ускладнює пакетну обробку та паралелізацію навчання. У RNN кожен елемент послідовності потрібно обробляти в послідовному порядку, що унеможливлює легку паралелізацію.

Використання механізмів уваги разом із цим обмеженням призвело до створення сучасних трансформерних моделей, які ми знаємо і використовуємо сьогодні, таких як BERT та OpenGPT3.

## Моделі трансформерів

Замість передачі контексту кожного попереднього передбачення на наступний етап оцінки, **моделі трансформерів** використовують **позиційні кодування** та увагу для захоплення контексту заданого входу в межах наданого текстового вікна. Зображення нижче показує, як позиційні кодування з увагою можуть захоплювати контекст у межах заданого вікна.

![Анімований GIF, що показує, як виконуються оцінки в моделях трансформерів.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Оскільки кожна вхідна позиція незалежно відображається на кожну вихідну позицію, трансформери можуть краще паралелізуватися, ніж RNN, що дозволяє створювати значно більші та виразніші мовні моделі. Кожна голова уваги може використовуватися для вивчення різних взаємозв’язків між словами, що покращує виконання задач обробки природної мови.

**BERT** (Bidirectional Encoder Representations from Transformers) — це дуже велика багатошарова трансформерна мережа з 12 шарами для *BERT-base* і 24 для *BERT-large*. Модель спочатку проходить попереднє навчання на великому корпусі текстових даних (WikiPedia + книги) за допомогою неконтрольованого навчання (передбачення замаскованих слів у реченні). Під час попереднього навчання модель засвоює значний рівень розуміння мови, який потім можна використовувати з іншими наборами даних за допомогою тонкого налаштування. Цей процес називається **трансферним навчанням**.

![Зображення з http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/uk/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Існує багато варіацій архітектур трансформерів, включаючи BERT, DistilBERT, BigBird, OpenGPT3 та інші, які можна налаштовувати. Пакет [HuggingFace](https://github.com/huggingface/) надає репозиторій для навчання багатьох із цих архітектур за допомогою PyTorch.

## Використання BERT для класифікації тексту

Давайте подивимося, як можна використовувати попередньо навчену модель BERT для вирішення нашої традиційної задачі: класифікації послідовностей. Ми будемо класифікувати наш оригінальний набір даних AG News.

Спочатку завантажимо бібліотеку HuggingFace та наш набір даних:


In [10]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...
Building vocab...


Оскільки ми будемо використовувати попередньо навчений BERT-модель, нам потрібно буде скористатися відповідним токенайзером. Спочатку ми завантажимо токенайзер, пов’язаний із попередньо навченою BERT-моделлю.

Бібліотека HuggingFace містить репозиторій попередньо навчених моделей, які можна використовувати, просто вказавши їхні назви як аргументи для функцій `from_pretrained`. Усі необхідні бінарні файли для моделі будуть автоматично завантажені.

Однак у деяких випадках вам може знадобитися завантажити власні моделі. У такому разі ви можете вказати каталог, який містить усі відповідні файли, включаючи параметри для токенайзера, файл `config.json` із параметрами моделі, бінарні ваги тощо.


In [11]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

Об'єкт `tokenizer` містить функцію `encode`, яку можна безпосередньо використовувати для кодування тексту:


In [15]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Тоді давайте створимо ітератори, які ми будемо використовувати під час навчання для доступу до даних. Оскільки BERT використовує власну функцію кодування, нам потрібно буде визначити функцію заповнення, схожу на `padify`, яку ми визначили раніше:


In [4]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

У нашому випадку ми будемо використовувати попередньо навчену модель BERT під назвою `bert-base-uncased`. Давайте завантажимо модель за допомогою пакету `BertForSequenceClassfication`. Це гарантує, що наша модель вже має необхідну архітектуру для класифікації, включаючи фінальний класифікатор. Ви побачите попередження, яке повідомляє, що ваги фінального класифікатора не ініціалізовані, і модель потребуватиме попереднього навчання - це абсолютно нормально, адже саме це ми збираємося зробити!


In [9]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at ./bert were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ./bert and

Тепер ми готові розпочати навчання! Оскільки BERT вже попередньо навчений, ми хочемо почати з досить малого коефіцієнта навчання, щоб не зруйнувати початкові ваги.

Всю важку роботу виконує модель `BertForSequenceClassification`. Коли ми викликаємо модель на навчальних даних, вона повертає як втрати, так і вихід мережі для вхідного мініпакету. Ми використовуємо втрати для оптимізації параметрів (`loss.backward()` виконує зворотний прохід), а `out` — для обчислення точності навчання, порівнюючи отримані мітки `labs` (обчислені за допомогою `argmax`) з очікуваними `labels`.

Для контролю процесу ми накопичуємо втрати та точність протягом кількох ітерацій і виводимо їх кожні `report_freq` цикли навчання.

Це навчання, ймовірно, займе досить багато часу, тому ми обмежуємо кількість ітерацій.


In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

report_freq = 50
iterations = 500 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 1.1254194641113282, Accuracy = 0.585
Loss = 0.6194715118408203, Accuracy = 0.83
Loss = 0.46665248870849607, Accuracy = 0.8475
Loss = 0.4309701919555664, Accuracy = 0.8575
Loss = 0.35427074432373046, Accuracy = 0.8825
Loss = 0.3306886291503906, Accuracy = 0.8975
Loss = 0.30340143203735354, Accuracy = 0.8975
Loss = 0.26139299392700194, Accuracy = 0.915
Loss = 0.26708646774291994, Accuracy = 0.9225
Loss = 0.3667240524291992, Accuracy = 0.8675


Ви можете побачити (особливо якщо збільшити кількість ітерацій і зачекати достатньо довго), що класифікація за допомогою BERT дає нам досить хорошу точність! Це тому, що BERT вже досить добре розуміє структуру мови, і нам потрібно лише доопрацювати фінальний класифікатор. Однак, через те, що BERT є великою моделлю, весь процес навчання займає багато часу і потребує значних обчислювальних ресурсів! (GPU, і бажано більше ніж один).

> **Примітка:** У нашому прикладі ми використовували одну з найменших попередньо навчених моделей BERT. Існують більші моделі, які, ймовірно, дадуть кращі результати.


## Оцінка продуктивності моделі

Тепер ми можемо оцінити продуктивність нашої моделі на тестовому наборі даних. Цикл оцінки дуже схожий на цикл навчання, але не забудьте перевести модель у режим оцінки, викликавши `model.eval()`.


In [10]:
model.eval()
iterations = 100
acc = 0
i = 0
for labels,texts in test_loader:
    labels = labels.to(device)-1      
    texts = texts.to(device)
    _, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc += torch.mean((labs==labels).type(torch.float32))
    i+=1
    if i>iterations: break
        
print(f"Final accuracy: {acc.item()/i}")

Final accuracy: 0.9047029702970297


## Основні висновки

У цьому розділі ми побачили, як легко взяти попередньо навчений мовний модель із бібліотеки **transformers** і адаптувати її до завдання класифікації тексту. Аналогічно, моделі BERT можна використовувати для вилучення сутностей, відповіді на запитання та інших завдань обробки природної мови.

Моделі трансформерів представляють сучасний передовий рівень у NLP, і в більшості випадків це має бути перше рішення, з якого ви починаєте експериментувати при впровадженні власних рішень NLP. Однак розуміння основних принципів рекурентних нейронних мереж, обговорених у цьому модулі, є надзвичайно важливим, якщо ви хочете створювати просунуті нейронні моделі.



---

**Відмова від відповідальності**:  
Цей документ був перекладений за допомогою сервісу автоматичного перекладу [Co-op Translator](https://github.com/Azure/co-op-translator). Хоча ми прагнемо до точності, будь ласка, майте на увазі, що автоматичні переклади можуть містити помилки або неточності. Оригінальний документ на його рідній мові слід вважати авторитетним джерелом. Для критично важливої інформації рекомендується професійний людський переклад. Ми не несемо відповідальності за будь-які непорозуміння або неправильні тлумачення, що виникають внаслідок використання цього перекладу.
